## Tasks to accomplish:
1. Get a list of topics and the links to their respective page (DONE)
2. Learn how to navigate each specific page (DONE)
3. Figure out how to deal with varying numbers of pages (DONE)
4. Create a full pipeline to automate the process
5. Export to CSV

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import pdb
import time
import json
from random import randint

## Task 1: Get a list of topics and the links to their respective pages

In [2]:
home_page = 'https://www.churchofjesuschrist.org'
topic_page = f'{home_page}/general-conference/topics'

var_names = ['speaker', 'title', 'topic', 'year', 'month', 'link']
conf_data = []
topics_and_links = {} #k: topic, v: link

topic_page = requests.get(topic_page)
topic_page_soup = BeautifulSoup(topic_page.content, 'html.parser')

script_soup = topic_page_soup.find_all("script")

topic_list_soup = None

for script in script_soup:
    if 'lists' in script.get_text():
        if "Jesus Christ" in script.get_text():
            topic_list_soup = script

topic_list_str = topic_list_soup.get_text().strip()

formatted_topic_list = topic_list_str[topic_list_str.find("{"):topic_list_str.find("}") - 2]
formatted_topic_list += "}"

topics_and_links = json.loads(formatted_topic_list)

topics_and_links.pop(" Jesus Christ")
topics_and_links.pop(" temples")

'/general-conference/topics/-temples?lang=eng&encoded=true'

# Task 2: Create a pipeline to go through all the talks on a given page (not accounting for multiple pages yet)

In [3]:
test_response = requests.get("https://www.google.com")

In [4]:
test_response.ok

True

In [5]:
# Functions
def create_page_soup(topic, link = ""):
    MAX_ATTEMPTS = 5
    num_attempts = 0
    
    if link == "":
        link = f'{home_page}{topics_and_links[topic]}'
    
    response = requests.get(link)
    
    while (not response.ok and num_attempts < MAX_ATTEMPTS):
        time.sleep(1)
        response = requests.get(link)
        num_attempts += 1
        print(f"Reattempting to connect with: {link}")
        
    if (response.ok):
        print(f"Successfully collected data from {link}")
        
    else:
        print(f"Failed to retrieve data from {link}.")
        print("Consider reviewing the webpage for broken links")
        
    topic_soup = BeautifulSoup(response.content, 'html.parser')
    return topic_soup

def scrape_talk_info(data_list, page_link, topic):
    topic_soup = BeautifulSoup(requests.get(page_link).content, 'html.parser')
    talks = topic_soup.find_all(class_= "lumen-tile__text-wrapper")
    for talk in talks:
        title = talk.find(class_ = "lumen-tile__title").get_text().strip()
        if "Video" in title:
            print("Skipping:", talk)
            continue
        print("\nTitle:", title)
        speaker = talk.find(class_ = "lumen-tile__content").get_text().strip()
        print("Speaker:", speaker)
        date = talk.find(class_ = "lumen-tile__metadata").get_text().strip()
        print("Date: ", date)
        date_list = date.split()
        
        if date_list[0] == "October":
            month = '10'
        else:
            month = '04'
        year = date_list[1]

        tag = talk.find('a')
        extension = tag.attrs['href']
        full_link = f'{home_page}{extension}'

        data_list.append((speaker, title, topic, month, year, full_link))
    
    return

def find_next_button(page_soup):
    next_button = page_soup.find_all(class_ = "pages-nav__list-item pages-nav__list-item--next")
    if len(next_button) > 0:
        return next_button[0].find("a").attrs['href']
    return None

In [6]:
topic_soup = create_page_soup("mercy")
talks = topic_soup.find_all(class_= "lumen-tile__text-wrapper")

Successfully collected data from https://www.churchofjesuschrist.org/general-conference/topics/mercy?lang=eng


In [7]:
topic = "mercy"
test_data = []
scrape_talk_info(test_data, f'{home_page}{topics_and_links[topic]}', topic)


Title: Do Justly, Love Mercy, and Walk Humbly with God
Speaker: Dale G. Renlund
Date:  October 2020

Title: By Union of Feeling We Obtain Power with God
Speaker: Sharon Eubank
Date:  October 2020

Title: Free Forever, to Act for Themselves
Speaker: D. Todd Christofferson
Date:  October 2014

Title: The Merciful Obtain Mercy
Speaker: Dieter F. Uchtdorf
Date:  April 2012

Title: The Miracle of the Atonement
Speaker: C. Scott Grow
Date:  April 2011

Title: Today
Speaker: Lance B. Wickman
Date:  April 2008

Title: The Tender Mercies of the Lord
Speaker: David A. Bednar
Date:  April 2005

Title: Remember How Merciful the Lord Hath Been
Speaker: Neal A. Maxwell
Date:  April 2004

Title: “Come, Follow Me”
Speaker: William W. Parmley
Date:  October 2003

Title: The Atonement: Our Greatest Hope
Speaker: James E. Faust
Date:  October 2001

Title: Our Father’s Plan
Speaker: Christoffel Golden Jr.
Date:  October 2001

Title: Welcome Home
Speaker: Keith B. McMullin
Date:  April 1999

Title: “He Ha

In [8]:
len(test_data)

22

# Task 3: Learn how to deal with varying numbers of pages

In [9]:
conf_data.clear()

In [10]:
topic = "agency"
link = f'{home_page}{topics_and_links[topic]}'
topic_soup = create_page_soup(topic)
talks = topic_soup.find_all(class_= "lumen-tile__text-wrapper")

if find_next_button(topic_soup) != None:
    while find_next_button(topic_soup) != None:
        scrape_talk_info(conf_data, link, topic)
        extension = find_next_button(topic_soup)
        link = f'{home_page}{extension}'
        topic_soup = create_page_soup(topic, link)
        
# The above logic will always skip pages that have no next button, but we still need that data
scrape_talk_info(conf_data, link, topic)

Successfully collected data from https://www.churchofjesuschrist.org/general-conference/topics/agency?lang=eng

Title: Let God Prevail
Speaker: Russell M. Nelson
Date:  October 2020

Title: Deep in Our Heart
Speaker: Douglas D. Holmes
Date:  April 2020

Title: Your Great Adventure
Speaker: Dieter F. Uchtdorf
Date:  October 2019

Title: Where Will This Lead?
Speaker: Dallin H. Oaks
Date:  April 2019

Title: Choose You This Day
Speaker: Dale G. Renlund
Date:  October 2018

Title: Turn to the Lord
Speaker: W. Christopher Waddell
Date:  October 2017

Title: Our Father’s Glorious Plan
Speaker: Weatherford T. Clayton
Date:  April 2017

Title: Fourth Floor, Last Door
Speaker: Dieter F. Uchtdorf
Date:  October 2016

Title: Choices
Speaker: Thomas S. Monson
Date:  April 2016

Title: The Holy Ghost
Speaker: Robert D. Hales
Date:  April 2016

Title: Opposition in All Things
Speaker: Dallin H. Oaks
Date:  April 2016

Title: Shipshape and Bristol Fashion: Be Temple Worthy—in Good Times and Bad Time

Successfully collected data from https://www.churchofjesuschrist.org/general-conference/topics/archive?topic=agency&lang=eng&page=4

Title: Making the Right Decisions
Speaker: L. Tom Perry
Date:  October 1979

Title: Faith, Courage, and Making Choices
Speaker: Dean L. Larsen
Date:  October 1978

Title: Privileges and Responsibilities of Sisters
Speaker: Spencer W. Kimball
Date:  October 1978

Title: Decision
Speaker: Eldred G. Smith
Date:  April 1978

Title: But Be Ye Doers of the Word
Speaker: L. Tom Perry
Date:  April 1977

Title: Spiritual Crocodiles
Speaker: Boyd K. Packer
Date:  April 1976

Title: The Right to Choose
Speaker: Henry D. Taylor
Date:  April 1976

Title: The Value of People
Speaker: Charles A. Didier
Date:  April 1976

Title: Using Our Free Agency
Speaker: Delbert L. Stapley
Date:  April 1975

Title: Understanding Who We Are Brings Self-Respect
Speaker: Harold B. Lee
Date:  October 1973

Title: “What Will a Man Give?”
Speaker: Mark E. Petersen
Date:  October 1973

Tit

In [11]:
len(conf_data)

163

In [12]:
conf_data.clear()

# Task 4: Put the pieces together for the full pipeline

In [13]:
# Set up
home_page = 'https://www.churchofjesuschrist.org'
topic_page = f'{home_page}/general-conference/topics'

var_names = ['speaker', 'title', 'topic', 'year', 'month', 'link']
conf_data = []
topics_and_links = {} #k: topic, v: link

topic_page = requests.get(topic_page)
topic_page_soup = BeautifulSoup(topic_page.content, 'html.parser')

script_soup = topic_page_soup.find_all("script")

topic_list_soup = None

# Find list of topics with urls
for script in script_soup:
    if 'lists' in script.get_text():
        if "Jesus Christ" in script.get_text():
            topic_list_soup = script

# Format that list to convert into dictionary
topic_list_str = topic_list_soup.get_text().strip()
formatted_topic_list = topic_list_str[topic_list_str.find("{"):topic_list_str.find("}") - 2]
formatted_topic_list += "}"
topics_and_links = json.loads(formatted_topic_list)

# Remove duplicates
topics_and_links.pop(" Jesus Christ")
topics_and_links.pop(" temples")

'/general-conference/topics/-temples?lang=eng&encoded=true'

## Make sure to run this cell and everything below if you need to update the conference data!

In [14]:
# Full pipeline
for topic, extension in topics_and_links.items():
    print(topic, f'{home_page}{extension}')
    link = f'{home_page}{extension}'
    topic_soup = create_page_soup(topic)
    talks = topic_soup.find_all(class_= "lumen-tile__text-wrapper")

    if find_next_button(topic_soup) != None:
        while find_next_button(topic_soup) != None:
            scrape_talk_info(conf_data, link, topic)
            extension = find_next_button(topic_soup)
            link = f'{home_page}{extension}'
            topic_soup = create_page_soup(topic, link)

    # The above logic will always skip pages that have no next button, but we still need that data
    scrape_talk_info(conf_data, link, topic)

 diversity https://www.churchofjesuschrist.org/general-conference/topics/-diversity?lang=eng&encoded=true
Successfully collected data from https://www.churchofjesuschrist.org/general-conference/topics/-diversity?lang=eng&encoded=true

Title: Hearts Knit in Righteousness and Unity
Speaker: Quentin L. Cook
Date:  October 2020

Title: By Union of Feeling We Obtain Power with God
Speaker: Sharon Eubank
Date:  October 2020
 friendship https://www.churchofjesuschrist.org/general-conference/topics/-friendship?lang=eng&encoded=true
Successfully collected data from https://www.churchofjesuschrist.org/general-conference/topics/-friendship?lang=eng&encoded=true

Title: I Believe in Angels
Speaker: Carlos A. Godoy
Date:  October 2020

Title: One in Christ
Speaker: Ulisses Soares
Date:  October 2018

Title: To the Rescue: We Can Do It
Speaker: Mervyn B. Arnold
Date:  April 2016

Title: You Are Not Alone in the Work
Speaker: Henry B. Eyring
Date:  October 2015

Title: Feed My Sheep
Speaker: Silvia H

Successfully collected data from https://www.churchofjesuschrist.org/general-conference/topics/archive?topic= love&lang=eng&page=2

Title: Eternal Life—to Know Our Heavenly Father and His Son, Jesus Christ
Speaker: Robert D. Hales
Date:  October 2014

Title: Our Personal Ministries
Speaker: Hugo E. Martinez
Date:  October 2014

Title: Living the Gospel Joyful
Speaker: Dieter F. Uchtdorf
Date:  October 2014

Title: Until We Meet Again
Speaker: Thomas S. Monson
Date:  April 2014

Title: Love—the Essence of the Gospel
Speaker: Thomas S. Monson
Date:  April 2014

Title: Sisterhood: Oh, How We Need Each Other
Speaker: Bonnie L. Oscarson
Date:  April 2014

Title: “I Have Given You an Example”
Speaker: Richard G. Scott
Date:  April 2014

Title: What Are You Thinking?
Speaker: W. Craig Zwick
Date:  April 2014

Title: The Power, Joy, and Love of Covenant Keeping
Speaker: Linda K. Burton
Date:  October 2013

Title: Drawing Closer to God
Speaker: Terence M. Vinson
Date:  October 2013

Title: We N

Successfully collected data from https://www.churchofjesuschrist.org/general-conference/topics/archive?topic= love&lang=eng&page=5

Title: A Season of Opportunity
Speaker: H. David Burton
Date:  October 1998

Title: To the Boys and to the Men
Speaker: Gordon B. Hinckley
Date:  October 1998

Title: Bear Record of Him
Speaker: Susan L. Warner
Date:  October 1998

Title: Healing Soul and Body
Speaker: Robert D. Hales
Date:  October 1998

Title: Come to Relief Society
Speaker: Virginia U. Jensen
Date:  October 1998

Title: Walking in the Light of the Lord
Speaker: Gordon B. Hinckley
Date:  October 1998

Title: Cultivating Divine Attributes
Speaker: Joseph B. Wirthlin
Date:  October 1998

Title: A Voice of Warning
Speaker: Henry B. Eyring
Date:  October 1998

Title: Opening the Windows of Heaven
Speaker: James E. Faust
Date:  October 1998

Title: The Power of Righteousness
Speaker: Richard G. Scott
Date:  October 1998

Title: Hope through the Atonement of Jesus Christ
Speaker: Neal A. Maxwe

Successfully collected data from https://www.churchofjesuschrist.org/general-conference/topics/archive?topic= love&lang=eng&page=8

Title: Church Welfare Services’ Basic Principles
Speaker: Marion G. Romney
Date:  April 1976

Title: Love Takes Time
Speaker: Marvin J. Ashton
Date:  October 1975

Title: We Can’t Do It Alone
Speaker: Robert D. Hales
Date:  October 1975

Title: Parents, Teach Your Children
Speaker: Paul H. Dunn
Date:  April 1974

Title: The Marriage That Endures
Speaker: Gordon B. Hinckley
Date:  April 1974

Title: “Behold Thy Mother”
Speaker: Thomas S. Monson
Date:  October 1973

Title: Thou Shalt Love Thy Wife with All Thy Heart
Speaker: Henry D. Taylor
Date:  October 1973

Title: You Shall Receive the Spirit
Speaker: Hartman Rector Jr.
Date:  October 1973

Title: Love and Forgive One Another
Speaker: O. Leslie Stone
Date:  October 1973

Title: What Manner of Men? “As I Am”
Speaker: Marion D. Hanks
Date:  April 1973

Title: “Go, and Do Thou Likewise”
Speaker: Robert L. S

Successfully collected data from https://www.churchofjesuschrist.org/general-conference/topics/archive?topic= missionary work&lang=eng&page=3

Title: The Essential Role of Member Missionary Work
Speaker: M. Russell Ballard
Date:  April 2003

Title: Stand in Your Appointed Place
Speaker: Thomas S. Monson
Date:  April 2003

Title: A Child and a Disciple
Speaker: Henry B. Eyring
Date:  April 2003

Title: Preparing for Missionary Service
Speaker: Daryl H. Garn
Date:  April 2003

Title: The Greatest Generation of Missionaries
Speaker: M. Russell Ballard
Date:  October 2002

Title: I’ll Go Where You Want Me to Go
Speaker: Dallin H. Oaks
Date:  October 2002

Title: To Men of the Priesthood
Speaker: Gordon B. Hinckley
Date:  October 2002

Title: They Pray and They Go
Speaker: Thomas S. Monson
Date:  April 2002

Title: Writing Gospel Principles in Our Hearts
Speaker: Walter F. González
Date:  October 2001

Title: The Returned Missionary
Speaker: L. Tom Perry
Date:  October 2001

Title: Sharing 

...

Successfully collected data from https://www.churchofjesuschrist.org/general-conference/topics/archive?topic=women&lang=eng&page=3

Title: Daughter of God
Speaker: Yoshihiko Kikuchi
Date:  April 1988

Title: Strengthening the Family
Speaker: Joanne B. Doxey
Date:  October 1987

Title: Lessons from Eve
Speaker: Russell M. Nelson
Date:  October 1987

Title: Overcoming Challenges along Life’s Way
Speaker: Joy F. Evans
Date:  October 1987

Title: To the Young Women of the Church
Speaker: Ezra Taft Benson
Date:  October 1986

Title: “If Thou Art Faithful”
Speaker: Gordon B. Hinckley
Date:  October 1984

Title: Restoring the Lost Sheep
Speaker: Joseph B. Wirthlin
Date:  April 1984

Title: Warmed by the Fires of Their Lives
Speaker: Barbara B. Smith
Date:  April 1984

Title: Choose the Good Part
Speaker: Marvin J. Ashton
Date:  April 1984

Title: Live Up to Your Inheritance
Speaker: Gordon B. Hinckley
Date:  October 1983

Title: A Season for Strength
Speaker: Barbara B. Smith
Date:  October 1

Successfully collected data from https://www.churchofjesuschrist.org/general-conference/topics/archive?topic=work&lang=eng&page=2

Title: Our Priceless Heritage
Speaker: Ezra Taft Benson
Date:  October 1976

Title: The Stone Cut without Hands
Speaker: Spencer W. Kimball
Date:  April 1976

Title: Family Preparedness
Speaker: Spencer W. Kimball
Date:  April 1976

Title: Church Welfare Services’ Basic Principles
Speaker: Marion G. Romney
Date:  April 1976

Title: The World’s Greatest Need
Speaker: Hartman Rector Jr.
Date:  October 1975

Title: The Time to Labor Is Now
Speaker: Spencer W. Kimball
Date:  October 1975

Title: Do It
Speaker: Robert L. Simpson
Date:  October 1975

Title: “The Gospel of Jesus Christ Is the Golden Door”
Speaker: Vaughn J. Featherstone
Date:  October 1973

Title: Church Welfare—Some Fundamentals
Speaker: Marion G. Romney
Date:  October 1973

Title: Altar, Tent, Well
Speaker: A. Theodore Tuttle
Date:  October 1972
worldliness https://www.churchofjesuschrist.org/ge

Successfully collected data from https://www.churchofjesuschrist.org/general-conference/topics/archive?topic=worthiness&lang=eng&page=2

Title: On Being Worthy
Speaker: Marvin J. Ashton
Date:  April 1989

Title: You Make a Difference
Speaker: Thomas S. Monson
Date:  April 1988

Title: To the Young Men of the Church
Speaker: Robert L. Backman
Date:  October 1980

Title: Salt of the Earth: Savor of Men and Saviors of Men
Speaker: Carlos E. Asay
Date:  April 1980

Title: The Blessing of Church Interviews
Speaker: N. Eldon Tanner
Date:  October 1978

Title: Worthy of Proper Recommendation
Speaker: N. Eldon Tanner
Date:  April 1978

Title: Our Responsibility to the Transgressor
Speaker: N. Eldon Tanner
Date:  October 1974
young adults https://www.churchofjesuschrist.org/general-conference/topics/young-adults?lang=eng&encoded=true
Successfully collected data from https://www.churchofjesuschrist.org/general-conference/topics/young-adults?lang=eng&encoded=true

Title: Meeting the Challenges of

Successfully collected data from https://www.churchofjesuschrist.org/general-conference/topics/archive?topic=youth&lang=eng&page=2

Title: “Put Your Shoulder to the Wheel”
Speaker: Neal A. Maxwell
Date:  April 1998

Title: Turning Hearts to the Family
Speaker: Margaret D. Nadauld
Date:  April 1998

Title: Living Worthy of the Girl You Will Someday Marry
Speaker: Gordon B. Hinckley
Date:  April 1998

Title: How Near to the Angels
Speaker: James E. Faust
Date:  April 1998

Title: In Harm’s Way
Speaker: Thomas S. Monson
Date:  April 1998

Title: Understanding Our True Identity
Speaker: Carol B. Thomas
Date:  April 1998

Title: Come unto Christ
Speaker: Margaret D. Nadauld
Date:  April 1998

Title: A Celestial Connection to Your Teenage Years
Speaker: Richard J. Maynes
Date:  October 1997

Title: “Behold the Man”
Speaker: Keith B. McMullin
Date:  October 1997

Title: Standing for Truth and Right
Speaker: M. Russell Ballard
Date:  October 1997

Title: The Savior Is Counting on You
Speaker: 

Successfully collected data from https://www.churchofjesuschrist.org/general-conference/topics/Zion?lang=eng

Title: Sisters in Zion
Speaker: Henry B. Eyring
Date:  October 2020

Title: Preparing for the Lord’s Return
Speaker: D. Todd Christofferson
Date:  April 2019

Title: Come to Zion
Speaker: D. Todd Christofferson
Date:  October 2008

Title: Zion in the Midst of Babylon
Speaker: David R. Stone
Date:  April 2006

Title: Come to Zion! Come to Zion!
Speaker: Keith B. McMullin
Date:  October 2002

Title: The Articles of Faith
Speaker: L. Tom Perry
Date:  April 1998

Title: The Kingdom’s Perfecting Pathway
Speaker: Dale E. Miller
Date:  April 1998

Title: Keep Walking, and Give Time a Chance
Speaker: Virginia H. Pearce
Date:  April 1997

Title: “Stand Ye in Holy Places”
Speaker: Lance B. Wickman
Date:  October 1994

Title: Becoming the Pure in Heart
Speaker: Spencer W. Kimball
Date:  April 1978

Title: Come: Let Israel Build Zion
Speaker: Bruce R. McConkie
Date:  April 1977

Title: Str

# Task 5: Export to CSV

In [15]:
conf_df = pd.DataFrame(conf_data, columns=var_names)

In [16]:
conf_df.head()

,speaker,title,topic,year,month,link
0,Quentin L. Cook,Hearts Knit in Righteousness and Unity,diversity,10,2020,https://www.churchofjesuschrist.org/general-co...
1,Sharon Eubank,By Union of Feeling We Obtain Power with God,diversity,10,2020,https://www.churchofjesuschrist.org/general-co...
2,Carlos A. Godoy,I Believe in Angels,friendship,10,2020,https://www.churchofjesuschrist.org/general-co...
3,Ulisses Soares,One in Christ,friendship,10,2018,https://www.churchofjesuschrist.org/general-co...
4,Mervyn B. Arnold,To the Rescue: We Can Do It,friendship,04,2016,https://www.churchofjesuschrist.org/general-co...


In [17]:
# Verify number of talks per topic. They're looking good for the most part, but just double check them
for topic in topics_and_links.keys():
    temp_df = conf_df[conf_df['topic'] == topic]
    print(topic, ":", len(temp_df))

 diversity : 2
 friendship : 37
 government : 10
 laws : 8
 Light of Christ : 6
 love : 357
 missionary work : 331
 morality : 169
 obedience : 387
 peace : 115
 preparation : 91
 prophets : 194
 religion : 8
 service : 422
Aaronic Priesthood : 10
abortion : 10
abuse : 21
accountability : 23
activation : 84
Adam and Eve : 6
addiction : 8
adversity : 281
agency : 163
angels : 9
anger : 10
animals : 1
apostasy : 27
Articles of Faith : 5
athletics : 2
Atonement : 265
attitude : 31
authority : 24
baptism : 10
belonging : 2
Bible : 6
bishops : 26
blessings : 62
Book of Mormon : 131
Brigham Young : 5
brotherhood : 14
character : 30
charity : 88
children : 152
Christianity : 8
Christmas : 1
Church activity : 1
Church attendance : 2
Church callings : 45
Church doctrine : 9
Church growth : 80
Church history : 36
Church leaders : 64
Church leadership : 1
Church meetings : 15
Church membership : 12
Church organization : 51
commandments : 117
commitment : 65
communication : 21
compassion : 30
conf

In [18]:
conf_df[conf_df['topic'] == '']

,speaker,title,topic,year,month,link


In [19]:
for col in conf_df.columns:
    print(col, conf_df[col].dtype)

speaker object
title object
topic object
year object
month object
link object


In [20]:
print(pd.__version__)

1.1.5


In [21]:
final_df = conf_df.copy()

In [22]:
final_df['speaker'] = final_df['speaker'].astype("string")
final_df['title'] = final_df['title'].astype('string')
final_df['topic'] = final_df['topic'].astype('string')
final_df['year'] = final_df['year'].astype(int)
final_df['month'] = final_df['month'].astype(int)
final_df['link'] = final_df['link'].astype('string')

In [23]:
for col in final_df.columns:
    print(col, final_df[col].dtype)

speaker string
title string
topic string
year int32
month int32
link string


In [29]:
conf_df.to_csv('talks_by_topic_dec_2020.csv', index=False)

In [30]:
conf_df_file = pd.read_csv("talks_by_topic_dec_2020.csv")

In [31]:
conf_df_file.head()

,speaker,title,topic,year,month,link
0,Quentin L. Cook,Hearts Knit in Righteousness and Unity,diversity,10,2020,https://www.churchofjesuschrist.org/general-co...
1,Sharon Eubank,By Union of Feeling We Obtain Power with God,diversity,10,2020,https://www.churchofjesuschrist.org/general-co...
2,Carlos A. Godoy,I Believe in Angels,friendship,10,2020,https://www.churchofjesuschrist.org/general-co...
3,Ulisses Soares,One in Christ,friendship,10,2018,https://www.churchofjesuschrist.org/general-co...
4,Mervyn B. Arnold,To the Rescue: We Can Do It,friendship,4,2016,https://www.churchofjesuschrist.org/general-co...
